In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import joblib

In [ ]:
def load_data():
  condo_df = pd.read_csv('../dataset/cleaned/bangkok-condo-dataset.csv')
  return condo_df

In [ ]:
condo_df = load_data()

In [ ]:
condo_df.shape

In [ ]:
condo_df = load_data()
condo_df.tail(5)

In [ ]:
condo_df.info()

In [ ]:
condo_df['Condo_Area_corr']

In [ ]:
# Drop unsed columns

cols = ['ID', 'Condo_NAME_EN', 'Condo_NAME_TH', 'Condo_link', 'All_Data', 'Condo_Area_corr']
condo_df = condo_df.drop(cols, axis=1)

In [ ]:
# Check missing value
(condo_df == '').sum()

In [ ]:
condo_df = condo_df.applymap(lambda x: np.nan if x == '' else x)

In [ ]:
# The missing data for floor feature can be filled by hand

condo_df.loc[condo_df['#_Floor'].isna(), :'#_Floor']

In [ ]:
condo_df.loc[1427, '#_Floor'] = 27
condo_df.loc[1428, '#_Floor'] = 5
condo_df.loc[[1427, 1428], :'#_Floor']

In [ ]:
# Boxplot Area_m2 (condo price per sqm)

condo_df['Area_m2'] = condo_df['Area_m2'].str.replace(',', '')
condo_df['Area_m2'] = condo_df['Area_m2'].fillna(-1).astype('int64').replace(-1, np.nan)

plt.figure(figsize=(5,10))
_, bp = condo_df.boxplot('Area_m2', return_type='both')

In [ ]:
# Q1 = condo_df['Area_m2'].quantile(0.25); # Q3 = condo_df['Area_m2'].quantile(0.75)
# IQR = Q3-Q1
# Outlier > Q3+1.5*IQR, Outlier < Q1-1.5*IQR

outliers = [flier.get_ydata() for flier in bp["fliers"]][0]
boxes = [box.get_ydata() for box in bp["boxes"]][0]
medians = [median.get_ydata() for median in bp["medians"]][0]
whiskers = [whiskers.get_ydata() for whiskers in bp["whiskers"]][0]

In [ ]:
medians

In [ ]:
# In which year that the outlier-condo were built?

outlier_condo = condo_df[condo_df['Area_m2'] > min(outliers)]['Year_built'].value_counts().sort_index()
plt.figure(figsize=(10,5))
plt.title('Built by year')
plt.xlabel('Year')
plt.ylabel('frequency')
ax = outlier_condo.plot.bar()
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.0025))

In [ ]:
# Distribution of Area_m2

plt.figure(figsize=(10,5))
plt.title('Histogram of Area_m2')
plt.xlabel('Area_m2')
ax = condo_df['Area_m2'].plot.hist(bins=30)


In [ ]:
# Distribution of Sale_Price_Inc[Year]

plt.figure(figsize=(10,5))
plt.title('Histogram of Sale_Price_Inc[Year]')
plt.xlabel('Sale_Price_Inc[Year]')
ax = condo_df['Sale_Price_Inc[Year]'].plot.hist(bins=50)

In [ ]:
# replace area_m2 and Sale_Price_Inc[Year] with median as the data are very skewed

condo_df['Area_m2'] = condo_df['Area_m2'].fillna(condo_df['Area_m2'].median())
condo_df['Sale_Price_Inc[Year]'] = condo_df['Sale_Price_Inc[Year]'].fillna(condo_df['Sale_Price_Inc[Year]'].median())

In [ ]:
# drop Rental_Yield_Inc[Year] as it has many missing value

condo_df = condo_df.drop('Rental_Yield_Inc[Year]', axis=1)

In [ ]:
# add kind of condo

condo_df['Kind'] = condo_df['#_Floor'].apply(lambda x: 'high rise' if x > 9 else 'low rise')

In [ ]:
condo_df['Address_TH']

In [ ]:
a = condo_df['Address_TH'].apply(lambda x: re.findall(r'(ถนน\s?[ก-๙]+\s?\d?)', x))

In [ ]:
a

In [ ]:
# add road

import re

condo_df['Road'] = condo_df['Address_TH'].apply(lambda x: re.findall(r'(ถนน\s?[ก-๙]+\s?\d?)', x))
condo_df['Road'] = condo_df['Road'].apply(lambda x: x[0] if len(x)> 0 else np.nan)
condo_df['Road'] = condo_df['Road'].str.replace(" ", "").str.strip()
condo_df['Road'] = condo_df['Road'].fillna(np.nan)
condo_df = condo_df.drop('Address_TH', axis=1)

In [ ]:
condo_df['Road'].value_counts().sum()

In [ ]:
# Visualize the correlation between features

correlation = round(condo_df.corr(numeric_only=True), 2)
cmap = sns.diverging_palette(220, 10, as_cmap=True)
plt.subplots(figsize=(11, 9))
sns.heatmap(correlation, cmap=cmap, linewidths=.5, annot=True, vmin=-1)

In [ ]:
condo_df = condo_df.drop('Sale_Price_Inc[Year]', axis=1)
condo_df = condo_df.drop('Sale_Price_Increment[Quarter]', axis=1)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(condo_df['Year_built'], condo_df['Sale_Price_Sqm'])

In [ ]:
condo_df['Road']

In [ ]:
# Feature engineering using one hot encoding

new_area = pd.get_dummies(condo_df['Condo_area'], dummy_na=True, prefix='Area_')
new_kind = pd.get_dummies(condo_df['Kind'], dummy_na=True, prefix='Kind_')
new_road = pd.get_dummies(condo_df['Road'], dummy_na=True, prefix='Road_')
condo_df = pd.concat([condo_df, new_area, new_kind, new_road], axis = 1)

condo_df = condo_df.drop('Condo_area', axis=1)
condo_df = condo_df.drop('Kind', axis=1)
condo_df = condo_df.drop('Road', axis=1)
condo_df.head()

In [ ]:
# create function clean_data(df) to do all cleaning
def clean_data(df):
  
  

  #drop unused columns
  cols = ['ID', 'Condo_NAME_EN', 'Condo_NAME_TH', 'Condo_link', 'All_Data', 'Condo_Area_corr']
  clean_df = df.drop(cols, axis=1)

  #replace '' with np.nan
  clean_df = clean_df.applymap(lambda x: np.nan if x == '' else x)

  #fill no.of Floor by hand
  clean_df.loc[1427, '#_Floor'] = 27
  clean_df.loc[1428, '#_Floor'] = 5

  #clean Area_m2 columns
  clean_df['Area_m2'] = clean_df['Area_m2'].str.replace(',', '')
  clean_df['Area_m2'] = clean_df['Area_m2'].fillna(-1).astype('int64').replace(-1, np.nan)

  #replace missing data with median as the data are very skewed
  clean_df['Area_m2'] = clean_df['Area_m2'].fillna(clean_df['Area_m2'].median())
  clean_df['Sale_Price_Inc[Year]'] = clean_df['Sale_Price_Inc[Year]'].fillna(clean_df['Sale_Price_Inc[Year]'].median())

  #drop Rental_Yield_Inc[Year] as it has many missing value
  clean_df = clean_df.drop('Rental_Yield_Inc[Year]', axis=1)

  #add kind of condo [>9 floor: high rise, else: low rise]
  clean_df['Kind'] = clean_df['#_Floor'].apply(lambda x: 'high rise' if x > 9 else 'low rise')

  #add road feature
  clean_df['Road'] = clean_df['Address_TH'].apply(lambda x: re.findall(r'(ถนน\s?[\u0E00-\u0E7F]+\s?\d?)', x))
  clean_df['Road'] = clean_df['Road'].apply(lambda x: x[0] if len(x)> 0 else np.nan)
  clean_df['Road'] = clean_df['Road'].str.replace(" ", "").str.strip()
  clean_df['Road'] = clean_df['Road'].fillna(np.nan)
  clean_df = clean_df.drop('Address_TH', axis=1)

  #drop Sale_Price_Inc[Year], Sale_Price_Increment[Quarter]
  clean_df = clean_df.drop('Sale_Price_Inc[Year]', axis=1)
  clean_df = clean_df.drop('Sale_Price_Increment[Quarter]', axis=1)

  #get_dummies
  new_area = pd.get_dummies(clean_df['Condo_area'], dummy_na=True, prefix='Area_')
  new_kind = pd.get_dummies(clean_df['Kind'], dummy_na=True, prefix='Kind_')
  new_road = pd.get_dummies(clean_df['Road'], dummy_na=True, prefix='Road_')
  clean_df = pd.concat([clean_df, new_area, new_kind, new_road], axis = 1)

  clean_df = clean_df.drop('Condo_area', axis=1)
  clean_df = clean_df.drop('Kind', axis=1)
  clean_df = clean_df.drop('Road', axis=1)

  return clean_df

In [ ]:
#run clean_data function
condo_df = load_data()
clean_df = clean_data(condo_df)
clean_df.head()

In [ ]:
y = condo_df['Sale_Price_Sqm'].values

In [ ]:
y

In [ ]:
# create function to split, extract feature and scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def create_dataset(df, random_state=42):
  y = df['Sale_Price_Sqm'].values
  X = df.drop('Sale_Price_Sqm', axis=1).copy()
  
  # y = MinMaxScaler().fit_transform(y.values.reshape(-1,1)).ravel()
  # y = MinMaxScaler().inverse_transform(y) to convert value back

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=random_state)

  # Minmax Scaler
  scaler = MinMaxScaler()

  # Fit scaler on training data and transform on other dataset to prevent information leakage
  X_train = scaler.fit_transform(X_train)
  X_val = scaler.transform(X_val)
  X_test = scaler.transform(X_test)

  return X_train, y_train, X_val, y_val, X_test, y_test, scaler

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, scaler = create_dataset(clean_df)
print(f"Train size: {X_train.shape[0]}")
print(f"Validation size: {X_val.shape[0]}")
print(f"Test size: {X_test.shape[0]}")

In [ ]:
# scaler ไว้สำหรับใช้ใน heroku
joblib.dump(scaler, "data_scaler.joblib")

In [ ]:
# create function to split, extract feature and scale 
# using StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler

def create_dataset_standard(df, random_state=42):
  y = df['Sale_Price_Sqm'].values
  X = df.drop('Sale_Price_Sqm', axis=1).copy()
  
  # y = MinMaxScaler().fit_transform(y.values.reshape(-1,1)).ravel()
  # y = MinMaxScaler().inverse_transform(y) to convert value back

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=random_state)

  # Minmax Scaler
  scaler = StandardScaler()

  # Fit scaler on training data and transform on other dataset to prevent information leakage
  # Scale only numerical data 
  num_cols = ['Year_built',	'Area_m2',	'#_Tower',	'#_Floor',	'Rental_Yield',	'Latitude',	'Longtitude',	'MinDist_Station']
  X_train = scaler.fit_transform(X_train[num_cols])
  X_val = scaler.transform(X_val[num_cols])
  X_test = scaler.transform(X_test[num_cols])

  return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
# from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.losses import mean_squared_error
from sklearn.model_selection import train_test_split
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import kerastuner as kt
from keras import backend as K

In [ ]:
# create function for RMSE loss evaluation
from sklearn.metrics import mean_squared_error as mse

def root_mean_squared_error(y_true, y_pred): # for keras
  result = K.sqrt(mean_squared_error(y_true, y_pred))
  return result

def rmse(y_true, y_pred): # for sklearn
  result = mse(y_true, y_pred, squared=False)
  return result

In [ ]:
# Add NN model

model = Sequential()
model.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#model.add(Dropout(0.25)) keep this for later optimization
model.add(Dense(30, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))
model.build(X_train.shape)
model.summary()

In [ ]:
# Complile model

model.compile(loss=root_mean_squared_error, optimizer='adam', metrics='mse')

In [ ]:
# Fit model
tf.config.run_functions_eagerly(True)

history = model.fit(X_train, y_train, epochs=100, batch_size=16,  verbose=1, validation_data=(X_val, y_val))

In [ ]:
# RMSE Loss
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
# Tune Model

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                     min_value=16,
                                     max_value=64,
                                     step=16),
                        activation='relu'))
       
        # drop_rate = hp.Choice('drop_rate_' + str(i), [0.0, 0.1, 0.2,0.3])
        # model.add(Dropout(rate=drop_rate))
    #model.add(Dense(1, activation='linear'))
    model.add(Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=root_mean_squared_error,
        metrics=['mse'])
    return model

In [ ]:
# tune NN

def nn_tunner(build_model):
  # Instantiate model tuner
  tuner = kt.RandomSearch(
      build_model,
      objective='mse',
      max_trials=10,
      seed = 42,
      directory = 'tuner')

  # Search summary
  print('\n############')
  print('\nTuning Summary')
  print(tuner.search_space_summary())

  # Initialize search space
  print('\n############')
  print('\nTuning Model')
  tuner.search(X_train, y_train,
              epochs=100,
              validation_data=(X_val, y_val))
  
  return tuner

In [ ]:
tuner = nn_tunner(build_model)

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# Tuned hyperparameters
tuner.get_best_hyperparameters()[0].values

In [ ]:
best_model.build(X_train.shape)
best_model.summary()

In [ ]:
X_val

In [ ]:
# RMSE of tuned model
y_hat = best_model.predict(X_val)
rmse(y_val, y_hat)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

rf = RandomForestRegressor(random_state=2020)

# Generate tuning parameters
n_estimators = np.arange(200,1000,200)
max_depth = np.arange(2,int(X_train.shape[1]/10) ,3)
min_samples_split = np.arange(2,5,1)
min_samples_leaf = np.arange(2,5,1)

rf_params = dict(n_estimators = n_estimators,  
               min_samples_split = min_samples_split, 
               max_depth = max_depth,
               min_samples_leaf = min_samples_leaf)

# Instantiate grid search with RMSE as scoring
rf_grid = GridSearchCV(rf, param_grid=rf_params, cv = 3, verbose = 1, n_jobs = -1, scoring='neg_root_mean_squared_error')

In [ ]:
rf_tuned = rf_grid.fit(X_train, y_train)

In [ ]:
rf_tuned.best_params_

In [ ]:
# average RMSE in cross validation
-np.nanmean(rf_grid.cv_results_['mean_test_score'])

In [ ]:
# Calculate RMSE from validation set
yhat_val = rf_tuned.best_estimator_.predict(X_val)
rmse(y_val, yhat_val)

In [ ]:
# export model
cls = rf_tuned.best_estimator_
joblib.dump(cls, 'rf.joblib')

In [ ]:
nn_best_model = joblib.load('rf.joblib')
predictions = nn_best_model.predict(X_val)
predictions

In [ ]:
from json import loads, dumps

# create function clean_data(df) to do all cleaning
def clean_data(df):

  #drop unused columns
  cols = ['ID', 'Condo_NAME_EN', 'Condo_NAME_TH', 'Condo_link', 'All_Data', 'Condo_Area_corr']
  clean_df = df.drop(cols, axis=1)

  #replace '' with np.nan
  clean_df = clean_df.applymap(lambda x: np.nan if x == '' else x)

  #fill no.of Floor by hand
  clean_df.loc[1427, '#_Floor'] = 27
  clean_df.loc[1428, '#_Floor'] = 5

  #clean Area_m2 columns
  clean_df['Area_m2'] = clean_df['Area_m2'].str.replace(',', '')
  clean_df['Area_m2'] = clean_df['Area_m2'].fillna(-1).astype('int64').replace(-1, np.nan)

  #replace missing data with median as the data are very skewed
  clean_df['Area_m2'] = clean_df['Area_m2'].fillna(clean_df['Area_m2'].median())
  clean_df['Sale_Price_Inc[Year]'] = clean_df['Sale_Price_Inc[Year]'].fillna(clean_df['Sale_Price_Inc[Year]'].median())

  #drop Rental_Yield_Inc[Year] as it has many missing value
  clean_df = clean_df.drop('Rental_Yield_Inc[Year]', axis=1)

  #add kind of condo [>9 floor: high rise, else: low rise]
  clean_df['Kind'] = clean_df['#_Floor'].apply(lambda x: 'high rise' if x > 9 else 'low rise')

  #add road feature
  clean_df['Road'] = clean_df['Address_TH'].apply(lambda x: re.findall(r'(ถนน\s?[\u0E00-\u0E7F]+\s?\d?)', x))
  clean_df['Road'] = clean_df['Road'].apply(lambda x: x[0] if len(x)> 0 else np.nan)
  clean_df['Road'] = clean_df['Road'].str.replace(" ", "").str.strip()
  clean_df['Road'] = clean_df['Road'].fillna(np.nan)
  clean_df = clean_df.drop('Address_TH', axis=1)

  #drop Sale_Price_Inc[Year], Sale_Price_Increment[Quarter]
  clean_df = clean_df.drop('Sale_Price_Inc[Year]', axis=1)
  clean_df = clean_df.drop('Sale_Price_Increment[Quarter]', axis=1)

  #get_dummies
  new_area = pd.get_dummies(clean_df['Condo_area'], dummy_na=True, prefix='Area_')
  new_kind = pd.get_dummies(clean_df['Kind'], dummy_na=True, prefix='Kind_')
  new_road = pd.get_dummies(clean_df['Road'], dummy_na=True, prefix='Road_')
  clean_df = pd.concat([clean_df, new_area, new_kind, new_road], axis = 1)

  clean_df = clean_df.drop('Condo_area', axis=1)
  clean_df = clean_df.drop('Kind', axis=1)
  clean_df = clean_df.drop('Road', axis=1)

  return clean_df

# create function to split, extract feature and scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def create_dataset(df, random_state=42):
  y = df['Sale_Price_Sqm'].values
  X = df.drop('Sale_Price_Sqm', axis=1).copy()
  
  # y = MinMaxScaler().fit_transform(y.values.reshape(-1,1)).ravel()
  # y = MinMaxScaler().inverse_transform(y) to convert value back

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=random_state)

  # Minmax Scaler
  scaler = MinMaxScaler()

  # Fit scaler on training data and transform on other dataset to prevent information leakage
  X_train = scaler.fit_transform(X_train)
  # print(X_val)
  X_val = scaler.transform(X_val)
  X_test = scaler.transform(X_test)

  return X_train, y_train, X_val, y_val, X_test, y_test, scaler

#run clean_data function
condo_df = load_data()
clean_df = clean_data(condo_df)
clean_df.head()

X_train, y_train, X_val, y_val, X_test, y_test, scaler = create_dataset(clean_df)

In [ ]:
input_json = [{
        "Condo_area": "Bang Kapi",
        "Year_built": 2011,
        "Area_m2": 6476.0,
        "#_Tower": 2,
        "#_Floor": 8.0,
        "Sale_Price_Sqm": 52065,
        "Rental_Yield": 4.86,
        "Latitude": 13.766348,
        "Longtitude": 100.649395,
        "MinDist_Station": 8256,
        "Kind": "low rise",
        "Road": "\u0e16\u0e19\u0e19\u0e40\u0e2a\u0e23\u0e35\u0e44\u0e17\u0e22"
    }]

input_df = pd.DataFrame.from_dict(input_json)

#get_dummies
new_area = pd.get_dummies(input_df['Condo_area'], dummy_na=True, prefix='Area_')
new_kind = pd.get_dummies(input_df['Kind'], dummy_na=True, prefix='Kind_')
new_road = pd.get_dummies(input_df['Road'], dummy_na=True, prefix='Road_')
input_df = pd.concat([input_df, new_area, new_kind, new_road], axis = 1)

input_df = input_df.drop('Condo_area', axis=1)
input_df = input_df.drop('Kind', axis=1)
input_df = input_df.drop('Road', axis=1)

scaler_model = joblib.load('data_scaler.joblib')
# print(scaler_model.feature_names_in_)
print(input_df)
full_df = pd.DataFrame(columns=scaler_model.feature_names_in_)
print(input_df.iloc[0,:])
full_df.loc[0] = input_df.iloc[0,:]
full_df = full_df.fillna(False)
print(full_df)

X = scaler_model.transform(full_df)
print(X)

rf_best_model = joblib.load('rf.joblib')
predictions = rf_best_model.predict(X)
print(predictions[0])